In [1]:
#import all the function needed to build model and manipulate
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import RMSprop
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import mido
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
#What does midi file look like
mid = MidiFile('TwoKids.mid') # a Mozart piece
print (type(mid))
print(mid.type)
print(mid.length)
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        if not msg.is_meta:
            if msg.channel == 5:
                if msg.type == 'note_on':
                    print(msg)

<class 'mido.midifiles.midifiles.MidiFile'>
1
191.99980799999977
Track 0: nameless
Track 1: Track 1
Track 2: Track 2
Track 3: Track 10


In [3]:
#function to organize the notes in [key velocity duration] format
def get_notes(path):
    mid = MidiFile(path) 
    notes = []

    time = float(0)
    prev = float(0)

    for msg in mid:
        ### this time is in seconds, not ticks
        time += msg.time
        #print (msg.time)
        if not msg.is_meta:
            ### only interested in piano channel
            try:
                if msg.channel == 1:
                    if msg.type == 'note_on':
                        #msg: note_on channel=0 note=59 velocity=100 time=1.0128208166666668
                        # note in vector form to train on
                        note = msg.bytes()
                        #print(note)
                        # only interested in the note and velocity. note message is in the form of [type, note, velocity]
                        note = note[1:3]
                        #print (note)
                        note.append(time-prev)
                        #print(note)
                        prev = time
                        notes.append(note)
            except:
                pass
    #each note should look like: [59, 100, 3.076924]
    return notes

In [8]:
#go through the country music directory, include all the files
import os
rootdir = 'midi/country/'
notes=[]
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print (file)
        path=rootdir+'/'+file
        notes+=get_notes(path)
#print(notes)

In [9]:
#check the notes
sum=0
for note in notes:
    sum+=note[2]
print(sum/len(notes))


0.42807844936572487


In [10]:
#normalize the data into 0, 1 
t = []
for note in notes:
    note[0] = (note[0]-24)/88
    note[1] = note[1]/127
    t.append(note[2])
max_t = max(t) # scale based on the biggest time of any note
for note in notes:
    note[2] = note[2]/max_t
#print(notes)

In [11]:
#organize the input and output data into a autoregressive way
X = []
Y = []
n_prev = 30
# n_prev notes to predict the (n_prev+1)th note
for i in range(len(notes)-n_prev):
    x = notes[i:i+n_prev]
    y = notes[i+n_prev]
    X.append(x)
    Y.append(y)
# save a seed to do prediction later
seed = notes[0:n_prev]

In [12]:
#Build model, train it 5 epoch (the sample we give is trained 500 epoches)
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(n_prev, 3), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, input_shape=(n_prev, 3), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(3))
model.add(Activation('linear'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='mse', optimizer='rmsprop')
model.summary()
model.fit(X, Y, batch_size=300, epochs=5, verbose=1)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 128)           67584     
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 195       
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total params: 117,187.0
Trainable params: 117,187.0
Non-trainable params: 0.0
_________________________________________________

In [16]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 

Saved model to disk


In [10]:
# load json and create model, once we have trained and saved one model, we can use it to generate music later on
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [13]:
#make predictions
prediction = []
x = seed
x = np.expand_dims(x, axis=0)

for i in range(3000):
    preds = model.predict(x)
    #print (preds)
    x = np.squeeze(x)
    x = np.concatenate((x, preds))
    x = x[1:]
    x = np.expand_dims(x, axis=0)
    preds = np.squeeze(preds)
    prediction.append(preds)
    
#turn data range back to the original range
for pred in prediction:
    pred[0] = int(88*pred[0] + 24)
    pred[1] = int(127*pred[1])
    pred[2] *= max_t
    # to reject values that will be out of range
    if pred[0] < 24:
        pred[0] = 24
    elif pred[0] > 102:
        pred[0] = 102
    if pred[1] < 0:
        pred[1] = 0
    elif pred[1] > 127:
        pred[1] = 127
    if pred[2] < 0:
        pred[2] = 0


In [14]:
#Save the predicted data into a midi file
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

for note in prediction:
    # 147 means note_on
    note = np.insert(note, 0, 147)
    bytes = note.astype(int)
    #print (note)
    msg = Message.from_bytes(bytes[0:3]) 
    time = int(note[3]/0.001025) # to rescale to midi's delta ticks. arbitrary value for now.
    msg.time = time
    track.append(msg)

mid.save('new_song.mid')